In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import joblib

# Set up device (use GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
# Load the data that includes the 'id' and 'date' columns
df = pd.read_csv('processed_shark_data.csv')

# Define feature columns to use
feature_columns = [
    'lat', 'lon', 'month', 'hour', 
    'speed_avg_roll', 'speed_std_roll', 'angle_avg_roll', 'angle_std_roll'
]
target_column = 'is_foraging'

# Scale the features
scaler = StandardScaler()
df[feature_columns] = scaler.fit_transform(df[feature_columns])
joblib.dump(scaler, 'pytorch_scaler.pkl') # Save the scaler

# Function to create sequences for each shark
def create_sequences(df, sequence_length=10):
    sequences = []
    targets = []
    # Group by shark ID to create sequences only within a single track
    for shark_id, group in df.groupby('id'):
        features = group[feature_columns].values
        labels = group[target_column].values
        
        if len(group) < sequence_length:
            continue
            
        for i in range(len(features) - sequence_length + 1):
            seq = features[i:i + sequence_length]
            target = labels[i + sequence_length - 1]
            sequences.append(seq)
            targets.append(target)
            
    return np.array(sequences), np.array(targets)

# Create the sequences
X_sequences, y_sequences = create_sequences(df, sequence_length=10)
print(f"Created {len(X_sequences)} sequences of length {X_sequences.shape[1]}.")

Created 8996 sequences of length 10.


In [3]:
class SharkDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = torch.tensor(sequences, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]

In [4]:
# Split the sequences into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_sequences, y_sequences, test_size=0.2, random_state=42, stratify=y_sequences
)

# Create Dataset instances
train_dataset = SharkDataset(X_train, y_train)
test_dataset = SharkDataset(X_test, y_test)

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"DataLoaders created with batch size {batch_size}.")

DataLoaders created with batch size 64.


In [5]:
class SharkLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.3):
        super(SharkLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True, # Important!
            dropout=dropout
        )
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_size, 1) # Output a single value
        
    def forward(self, x):
        # LSTM returns output and hidden states
        # We only need the output from the last time step
        lstm_out, _ = self.lstm(x)
        last_time_step_out = lstm_out[:, -1, :]
        
        out = self.dropout(last_time_step_out)
        out = self.fc(out)
        return out

In [6]:
# Initialize the model
input_size = X_train.shape[2] # Number of features
model = SharkLSTM(input_size=input_size).to(device)

# Define loss function and optimizer
# BCEWithLogitsLoss is numerically stable and includes the sigmoid activation
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50
print("Starting training...")
for epoch in range(epochs):
    model.train() # Set the model to training mode
    total_loss = 0
    for sequences, targets in train_loader:
        # Move data to the configured device
        sequences, targets = sequences.to(device), targets.to(device)
        
        # Forward pass
        outputs = model(sequences).squeeze()
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    avg_loss = total_loss / len(train_loader)
    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')

print("Training complete!")

Starting training...
Epoch [5/50], Loss: 0.6030
Epoch [10/50], Loss: 0.5955
Epoch [15/50], Loss: 0.5853
Epoch [20/50], Loss: 0.5770
Epoch [25/50], Loss: 0.5662
Epoch [30/50], Loss: 0.5548
Epoch [35/50], Loss: 0.5358
Epoch [40/50], Loss: 0.5176
Epoch [45/50], Loss: 0.4988
Epoch [50/50], Loss: 0.4657
Training complete!


In [7]:
model.eval() # Set the model to evaluation mode
all_preds = []
all_targets = []

with torch.no_grad(): # No need to calculate gradients for evaluation
    for sequences, targets in test_loader:
        sequences = sequences.to(device)
        outputs = model(sequences).squeeze()
        
        # Apply sigmoid to get probabilities, then threshold at 0.5 for class labels
        preds = (torch.sigmoid(outputs) >= 0.5).cpu().numpy()
        
        all_preds.extend(preds)
        all_targets.extend(targets.numpy())

# Calculate and print the final results
accuracy = (np.array(all_preds) == np.array(all_targets)).mean()
print(f"\n--- Final Model Evaluation ---")
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:")
print(classification_report(all_targets, all_preds, target_names=['Traveling', 'Foraging']))

# Save the model
torch.save(model.state_dict(), 'shark_pytorch_lstm_model.pth')
print("\nPyTorch model saved as 'shark_pytorch_lstm_model.pth'")


--- Final Model Evaluation ---
Model Accuracy: 66.00%

Classification Report:
              precision    recall  f1-score   support

   Traveling       0.73      0.82      0.77      1264
    Foraging       0.40      0.28      0.33       536

    accuracy                           0.66      1800
   macro avg       0.56      0.55      0.55      1800
weighted avg       0.63      0.66      0.64      1800


PyTorch model saved as 'shark_pytorch_lstm_model.pth'
